# path AdamicAdar

In [1]:
import sys
import re, os
import json 
import requests
from py2neo import Node, Relationship, Graph, NodeMatcher, RelationshipMatcher
from py2neo import NodeMatcher, RelationshipMatcher
print(os.getcwd())
print(sys.version)

/home/aid/Github/kg
3.7.13 (default, Mar 29 2022, 02:18:16) 
[GCC 7.5.0]


In [11]:
graph = Graph("http://192.168.200.155:7474", auth=("neo4j", "aid_neo4j"))
nodematcher= NodeMatcher(graph)
relamatcher = RelationshipMatcher(graph)

match_str = "MATCH (na:drug)-[*1..10]->(nb) WHERE na.name=~'(?i).*布地奈德.*' RETURN (na)-[*1..10]->(nb)"
match_str = '''MATCH p=(n1:drug)-[r1]-(n2)-[r2]-(n3:disease {name:'腹痛'})
WHERE not (n1)-[]-(n3)
WITH n1.name as Drug, n3.name as disease, n2.name as NEighborName, 
1/log(size((n2)-[]-())) as AdamicAdarScore
RETURN Drug, disease, sum(AdamicAdarScore) as TotalAdamicAdarScore
ORDER BY TotalAdamicAdarScore DESC'''
results = graph.run(match_str)

# nodes = [] 
# for i in results:
#     node_str = re.sub("Path|Node|name=", "", str(i))
#     node_str = re.sub("[\!\%\[\]\,\。\{\}\-\:\'\(\)\>]", " ", node_str).split(' ')
#     node_str = [i for i in node_str if len(i)>1]
#     if len(node_str) == 7:
#         nodes.append(node_str[1])
#         nodes.append(node_str[6])

# nodes = list(set(nodes))
# print(nodes)

for i in results:
    # if '布地奈德' in i[0]:
    #     print(i) 
    print(i)


'绿豆'	'腹痛'	50.3503490555612
'鹅肉'	'腹痛'	12.934482402590861
'吡喹酮片'	'腹痛'	12.437623078521215
'消癌平片'	'腹痛'	12.323512804074724
'蜂蜜'	'腹痛'	11.053358615867527
'阿苯达唑片'	'腹痛'	10.845165095674286
'苯丁酸氮芥片'	'腹痛'	10.695682064488208
'布美他尼片'	'腹痛'	8.619475933764804
'复方谷氨酰胺肠溶胶囊'	'腹痛'	8.087657843683504
'仁青芒觉'	'腹痛'	8.03891304601614
'季德胜蛇药片'	'腹痛'	8.03891304601614
'诺氟沙星胶囊'	'腹痛'	7.94867092621497
'枯草杆菌二联活菌肠溶胶囊'	'腹痛'	7.309833053504621
'盐酸左氧氟沙星胶囊'	'腹痛'	7.200454211794423
'小儿暖脐膏'	'腹痛'	6.5440230254860605
'十香暖脐膏'	'腹痛'	6.24704882111269
'复合乳酸菌胶囊'	'腹痛'	5.990536103137928
'复方猴头颗粒'	'腹痛'	5.898106845919172
'肠内营养混悬液SP'	'腹痛'	5.779840585798137
'盐酸屈他维林片'	'腹痛'	5.490719146076263
'十一味金色丸'	'腹痛'	5.476097831601684
'豆腐'	'腹痛'	5.398491804187312
'强力康颗粒'	'腹痛'	5.356667110662766
'头孢克肟颗粒'	'腹痛'	5.338866146428409
'醋酸泼尼松片'	'腹痛'	5.283103582777748
'氯霉素片'	'腹痛'	5.153715642876117
'消癌平胶囊'	'腹痛'	4.776248634101713
'小金胶囊'	'腹痛'	4.509167081177786
'注射用硫酸长春新碱'	'腹痛'	4.339295562481647
'奥美拉唑肠溶胶囊'	'腹痛'	4.262222551538609
'红金消结片'	'腹痛'	3.9958989874786006
'盐酸克林霉素棕榈酸酯分散片'

# TuckER&&TransE

In [1]:
import pandas as pd
import numpy as np

import pymysql
import configparser

import ampligraph
from ampligraph.latent_features import TransE, ComplEx
from ampligraph.utils import save_model, restore_model, create_tensorboard_visualizations
from ampligraph.evaluation import mr_score, mrr_score, hits_at_n_score, evaluate_performance, train_test_split_no_unseen

import tensorflow as tf

from scipy.special import expit

/home/aid/miniconda3/envs/py37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/aid/miniconda3/envs/py37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/aid/miniconda3/envs/py37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/aid/miniconda3/envs/py37/lib/python3.7/site-packages

In [2]:
from ampligraph.discovery import query_topn

model = restore_model('/data/nlp_models/TransE-small.pkl')

# data = get_mysql_data("SELECT entity,relation FROM triple WHERE  o_type IN ('disease','symptom','check', 'drug') LIMIT 100;")
# data_dt = {}
# for i in data:
#     data_dt[i[0]] = i[1]

# for k,v in data_dt.items():
#     print(query_topn(model, top_n=10, head=k, relation=v, tail=None))


print(query_topn(model, top_n=50, head='哮喘', relation='Symptom_Disease', tail=None))


2022-05-19 09:05:50.738534: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-05-19 09:05:50.761978: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3699850000 Hz
2022-05-19 09:05:50.762947: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55eb53b9d880 executing computations on platform Host. Devices:
2022-05-19 09:05:50.762960: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2022-05-19 09:05:52.021571: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not via TF_XLA_FLAGS) 

(array([['哮喘', 'Symptom_Disease', '脱症'],
       ['哮喘', 'Symptom_Disease', '肾小球基底膜蛾噬现象'],
       ['哮喘', 'Symptom_Disease', '帕里诺眼-腺综合征'],
       ['哮喘', 'Symptom_Disease', '腹部“气串样”肿块'],
       ['哮喘', 'Symptom_Disease', '剧烈运动后呕吐'],
       ['哮喘', 'Symptom_Disease', '脓疱性痤疮'],
       ['哮喘', 'Symptom_Disease', '菌尿'],
       ['哮喘', 'Symptom_Disease', '小儿双下肢蜷曲'],
       ['哮喘', 'Symptom_Disease', '额头长痘'],
       ['哮喘', 'Symptom_Disease', '牙龈深红或暗红色'],
       ['哮喘', 'Symptom_Disease', '扁桃体小或缺如'],
       ['哮喘', 'Symptom_Disease', '饮水呛咳'],
       ['哮喘', 'Symptom_Disease', '食管恶性病变'],
       ['哮喘', 'Symptom_Disease', '鼻衂'],
       ['哮喘', 'Symptom_Disease', '新生儿皮肤薄'],
       ['哮喘', 'Symptom_Disease', '尾骨压痛及异常活动'],
       ['哮喘', 'Symptom_Disease', '反复肺不张'],
       ['哮喘', 'Symptom_Disease', '上半身代偿性多汗症'],
       ['哮喘', 'Symptom_Disease', '新生儿心力衰竭'],
       ['哮喘', 'Symptom_Disease', '震荡性神经受累'],
       ['哮喘', 'Symptom_Disease', '真皮侵袭性生长'],
       ['哮喘', 'Symptom_Disease', '心悸伴高血压'],
       ['哮喘', 'Symptom_Di